In [23]:
from tqdm.auto import tqdm
import requests
from elasticsearch import Elasticsearch
import tiktoken

## Question 1. Running Elastic

In [15]:
requests.get("http://localhost:9200").json()["version"]["build_hash"]

'42f05b9372a9a4a470db3b52817899b99a76ee73'

## Question 2. Indexing the data

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
len(documents)

948

In [3]:
es_client = Elasticsearch("http://localhost:9200")
# make sure it's empty
es_client.indices.get_alias()

ObjectApiResponse({})

In [4]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [5]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Question 3. Searching: score for top record

In [6]:
query = "How do I execute a command in a running docker container?"

search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                    }
                },
                "filter": {"term": {"course": "machine-learning-zoomcamp"}},
            }
        }
    }
response = es_client.search(index=index_name, body=search_query)

In [7]:
response["hits"]["max_score"]

84.050095

## Question 4. Filtering: 3rd document 

In [9]:
response["hits"]["hits"][2]["_source"]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

## Question 5. Prompt length 

In [20]:
question = "How do I execute a command in a running docker container?"
context_template = """
Q: {question}
A: {text}
""".strip()


def get_context(hits: dict):
    context = "\n\n".join(
        context_template.format(
            question=hit["_source"]["question"],
            text=hit["_source"]["text"]
        )
        for hit in hits
    )
    return context


prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{get_context(hits=response["hits"]["hits"])}
""".strip()

In [22]:
len(prompt)

1462

## Question 6. Number of tokens (1 point)

In [24]:
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))

322